**Project 2 Phase 2**<br>
Group 8
<br>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Peinthor</td>
    <td style = "text-align: left">11815592</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Michael</td>
    <td style = "text-align: left">Weikl</td>
    <td style = "text-align: left">1154652</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Uros</td>
    <td style = "text-align: left">Zivanovic</td>
    <td style = "text-align: left">12032271</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Maximilian</td>
    <td style = "text-align: left">Hofmann</td>
    <td style = "text-align: left">12019987</td>
  </tr>
  
</table>
<br>

# Basic stuff

In [1]:
from typing import Union

import numpy as np
import numpy.linalg as lin
import scipy.optimize

In [2]:
class Problem():
    def f(self, x):
        raise NotImplementedError("f() is not implemented for this Problem")
    
    def grad_f(self, x):
        raise NotImplementedError("grad_f() is not implemented for this Problem")
            
    def hessian(self, x):
        raise NotImplementedError("hessian() is not implemented for this Problem")      

    def equality_constraints(self, x):
        raise NotImplementedError("equality_constraints() is not implemented for this Problem")

    def grad_eqc(self, x):
        raise NotImplementedError("grad_eqc() is not implemented for this Problem")

    def inequality_constraints(self, x):
        raise NotImplementedError("inequality_constraints() is not implemented for this Problem")

    def grad_ieqc(self, x):
        raise NotImplementedError("grad_ieqc() is not implemented for this Problem")
        

In [3]:
class PolynomialProblem(Problem):
        @property
        def actual_minima(self):
            raise NotImplementedError("actual_minima() is not implemented for this Problem")      


############################################################
#                Polynomial definitions
############################################################

class Rosenbrock(PolynomialProblem):
    def f(self, x):
        return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

    def grad_f(self, x):
        return np.array([-400*x[0]*(x[1] - x[0]**2) - 2*(1 - x[0]),
                         200*(x[1] - x[0]**2)])

    def hessian(self, x):
        return np.array([[-400*(x[1] - 3*x[0]**2) + 2, 
                          -400*x[0]],
                         [-400*x[0], 
                          200]])
    
    def inequality_constraints(self, x):
        x = x.reshape((-1, 1))
        return np.array([-x[0] ** 2 - x[1] ** 2, x[1]]), np.array([[-1], [0]])
        # return np.array([-1, -1]), np.array([0, 1]), np.array([-1, 0])

    def grad_ieqc(self, x):
        return np.array([[-2 * x[0], -2 * x[1]], [0, 0]])
        # return np.array([0, 0]), np.array([-2, -2]), np.array([-1, 0])
    
    @property
    def actual_minima(self):
        return np.array([1,1])


#this is the function below rosenbrock defined in the project description
class Function2(PolynomialProblem):
    def f(self, x):
        return 150*(x[0] * x[1])**2 + (0.5 * x[0] + 2 * x[1] - 2)**2
    
    def grad_f(self, x):
        return np.array([x[0] * (300 * x[1]**2 + 0.5) + 2 * x[1] - 2, 300*x[0]**2 * x[1] + 2 * x[0] + 8 * x[1] - 8])

    def hessian(self, x):
        return np.array([[300 * x[1]**2 + 0.5, 600 * x[0]*x[1] + 2],[600 * x[0]*x[1] + 2, 300 * x[0]**2 + 8]])

    #self calculated minima
    @property
    def actual_minima(self):
        return np.array([[0,1],[4,0]])

# Implementations

## Simplex

In [4]:
def simplex_method(linear_problem: Union[np.ndarray, None], constraints: np.ndarray, constraint_rhs: np.ndarray):
    def swap(variables: list, exits_idx: int, enters_idx: int) -> None:
        """
        In-/excluding respective vectors into/from the basis.
        """
        tmp = variables[exits_idx]
        variables[exits_idx] = variables[enters_idx]
        variables[enters_idx] = tmp
    
    constraint_rhs = constraint_rhs.reshape((-1, 1))
    n_constraints = constraints.shape[0]  # Number of basis vectors
    n_orig_variables = constraints.shape[1]  # Number of original_variables in each constraint
    n_variables = constraints.shape[1]  # Number of variables in each constraint

    tableau = np.concatenate((constraints, np.eye(n_constraints)), axis=1)  # Extended matrix of the system by slack constraints
    x = np.concatenate((np.zeros([n_variables, 1]), constraint_rhs))  # Right parts of the constraints
    
    non_base_weigths = np.zeros((n_variables, 1)) if linear_problem is None else linear_problem
    base_weights = np.ones((n_constraints, 1)) if linear_problem is None else np.zeros((n_constraints, 1))
    weights = np.vstack((non_base_weigths, base_weights))
    
    variable_idx = list(range(len(weights)))  # Indices for all variables
    priv_x_B, x_B = None, None


    while True:
        basis_idx = variable_idx[n_variables:]
        basis = tableau.T[basis_idx].T
        basis_inv = lin.inv(basis)
        priv_x_B = x_B
        x_B = x[basis_idx]
        c_B = weights[basis_idx]
        lam = lin.inv(basis.T) @ c_B

        non_basis_idx = variable_idx[:n_variables]
        c_N = weights[non_basis_idx]
        n_T = tableau.T[non_basis_idx]
        s_N = c_N - (n_T @ lam)

        if np.all(s_N >= 0) or (x_B is not None and priv_x_B is not None and np.sum(priv_x_B - x_B) == 0):
            print("Optimal solution found!!!")
            solution = None if linear_problem is None else c_B.T @ basis_inv @ constraint_rhs
            return x[:n_orig_variables].flatten(), x[n_orig_variables:].flatten(), solution

        # since here must be at least one s_q < 0, we take the argmin to get the index of the smallest value.
        # q_idx = np.argmin(s_N)  # Fetch the idx in the non basis idx set
        # q = non_basis_idx[q_idx]  # Fetch the variable_idx
        # d = basis_inv @ tableau.T[q].T
        # d = np.reshape(d, (-1, 1))

        d = basis_inv @ tableau
        n_q = np.vstack((-d.T[:n_variables].T, np.identity(n_variables)))

        change = (weights.T @ n_q) / np.linalg.norm(n_q, axis=0)
        change[(s_N >= 0).T] = float('inf')
        q_idx = np.argmin(change)
        q = non_basis_idx[q_idx]

        if np.all(d <= 0):
            print("Problem is unbound!!!")
            return None, None, None
        d_q = np.reshape(d[:, q], (-1, 1))
        piv_weigths = x_B / d_q
        p_idx = np.argmin(piv_weigths)
        x_q = piv_weigths[p_idx]
        x[basis_idx] = x_B - (d_q * x_q)
        x[q] = x_q  # set entering variable vaLue

        # In-/excluding respective vectors into/from the basis:
        swap(variables=variable_idx, exits_idx=p_idx + n_variables, enters_idx=q_idx)

## Active Set

In [5]:
def compute_learning_rate(A, p, x, w, b):
    numerator = b - (A @ x)
    denominator = A @ p
    possible_alphas = numerator / denominator

    idx_gtz = np.where(denominator > 0)[0].tolist()

    if len(idx_gtz) == 0:
        return 1, None
    else:
        blocking = []
        for idx in idx_gtz:
            if idx not in w:
                blocking.append(idx)

        if len(blocking) == 0:
            return 1, None
        else:
            possible_alphas = possible_alphas[blocking]
            min_alpha = np.argmin(possible_alphas)
            if len(min_alpha.shape) != 0:
                min_alpha = min_alpha[0]

            if possible_alphas[min_alpha] < 1:
                return possible_alphas[min_alpha], min_alpha
            else:
                return 1, None


def compute_descent_direction_and_lagrangien(G, x, c, A, w):
    working_A = A[w]
    size_of_zeros = working_A.shape[0]

#     problem = np.concatenate((
#         np.concatenate((G, working_A.T), axis=1),
#         np.concatenate((working_A, np.zeros((size_of_zeros, size_of_zeros))), axis=1)),
#         axis=0)
    problem = np.vstack((
        np.hstack((G, working_A.T)),
        np.hstack((working_A, np.zeros((size_of_zeros, size_of_zeros))))))
    g_k = (G @ x) + c
    rhs = np.vstack((-g_k, np.zeros((size_of_zeros, 1))))

    result = np.linalg.lstsq(problem, rhs)[0]

    descent_direction = result[0:len(G)]
    lagrangiens = result[len(G):]

    return descent_direction, lagrangiens


def active_set_method(G, c, A, b, inequalities=True, x0=None, steps=1000):
    if x0 is None:
        x, _, _ = simplex_method(None, A, b)
        print(f"No starting point given: Found one feasible point at '{x}'")
    else:
        x = np.copy(x0)
        
    c = c.reshape((-1, 1))
    b = b.reshape((-1, 1))

    # equalities are always active
    w: list = np.where(A @ x == b)[0].tolist() if inequalities else [i for i in range(A.shape[0])]

    for k in range(steps + 1):
        p, lambdas = compute_descent_direction_and_lagrangien(G=G, x=x, c=c, A=A, w=w)

        if np.all(np.abs(p) <= 1e-15):
            if np.all(lambdas >= 0):
                return x, k
            else:
                j = np.argmin(lambdas)
                w_j = w[j]
                while w_j in w:
                    w.remove(w_j)

        else:
            alpha, blocking_idx = compute_learning_rate(A, p, x, w, b)
            x = x + alpha * p

            # Are there blocking constraints
            # http://lendek.net/teaching/OPT/qp.pdf
            if alpha < 1:
                w.append(blocking_idx)

    return None, k

## SQP

In [6]:
def e_i(size, index):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr


# these return the function that computes the gradient, which can then in turn be called to compute the gradient
def approx_grad(f, e=1.1e-8):
    def grad_f(x):
        if x.size == 1:
            return (f(x + e) - f(x)) / e
        return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])

    return grad_f


def approx_hessian(f, e=1.1e-8):
    def hessian_f(x):
        if x.size == 1:
            return (f(x + 2 * e) - 2 * f(x + e) + f(x)) / e ** 2
        return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
            x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
            x)) / e ** 2 for j in range(x.size)] for i in range(x.size)])

    return hessian_f


# f_hess_k, x_k, x_k1, lam_k, lam_k1, a_k, f_grad
def approx_hessian_bfgs(B_k, xk, x1, lam1, c, fp, alpha, pk):
    """
    Approximate Hessian based on first derivitive using previous values.
    :param f_grad: first derivitive of f
    :param B_k: current hessian
    :param x_1: x_{k+1}
    :param x: x_{k}
    :return: approximated hessian
    """

    s_k = alpha * pk

    fpk = fp(xk).reshape((-1, 1))
    fpk1 = fp(x1).reshape((-1, 1))
    ck = c(xk)[0].reshape((-1, 1))
    ck1 = c(x1)[0].reshape((-1, 1))
    lam1 = lam1.reshape((-1, 1))

    lg = fpk - ck.T @ lam1
    lg1 = fpk1 - ck1.T @ lam1

    y_k = lg1 - lg

    return B_k - (B_k * s_k * np.transpose(s_k) * B_k) / \
           (np.transpose(s_k) * B_k * s_k) + \
           (y_k * np.transpose(y_k)) / (np.transpose(y_k) * s_k)


def compute_lagrangien_sqp(H, A, p, fp, c):
    working_A = np.copy(A)
    size_of_zeros = working_A.shape[0]
    size_of_ones = working_A.shape[1]

    problem = np.vstack((
        np.hstack((H, -working_A)),
        np.hstack((working_A.T, np.zeros((size_of_zeros, size_of_zeros)))),
        np.hstack((np.eye(size_of_ones), np.zeros((len(p), size_of_zeros))))))

    rhs = np.concatenate((-fp, -c, p), axis=0)

    result = np.linalg.lstsq(problem, rhs)
    result = result[0]

    lagrangiens = result[len(H):]

    return lagrangiens.squeeze()


def choose_mu(H, fpk, pk, ck, mu=None, rho=0.8, mu_delta=0, sigma=1):
    nominator = fpk.T @ pk + (sigma / 2) * pk.T @ H @ pk
    denominator = (1 - rho) * np.linalg.norm(ck, 1)

    lower_bound = nominator / denominator

    if mu is None or mu < lower_bound:
        mu = lower_bound + mu_delta

    return mu


def l1_merit_function(f, x, c, mu):
    fx = f(x)
    cx, _ = c(x)

    return (fx + mu * np.sum(np.maximum(-cx, 0))).squeeze()


def choose_alpha(f, fpk, x, c, ck, pk, eta, mu, tau):
    alpha = 1
    x = x.reshape((-1, 1))

    while True:
        compare_value = x + (alpha * pk)
        compare_merit_value = l1_merit_function(f, compare_value, c, mu)
        base_merit = l1_merit_function(f, x, c, mu)
        directional_derivate = fpk.T @ pk - mu * np.linalg.norm(ck, ord=1)

        base = base_merit + eta * alpha * directional_derivate

        if compare_merit_value <= base:
            return alpha

        tau_alpha = np.random.uniform(low=0, high=tau)
        alpha = alpha * tau_alpha


def stop_crit(grad_f, xk, x0, current_iteration, tol: float = 1e-8, max_iter: int = 1000):
    if current_iteration > max_iter:
        return True
    elif np.linalg.norm(grad_f(xk)) <= tol * np.linalg.norm(grad_f(x0)):
        return True
    return False


def sqp_method(f, f_grad=None, f_hessian=None, ieq_conststraints=None, grad_ieq=None, x0=None, l0=None, steps=1000,
               mu_delta=1e-2, eta=0.3, tau: float = 0.9):
    if f_grad is None:
        f_grad = approx_grad(f)
    else:
        f_grad = f_grad

    f_hess_k = np.eye(x0.shape[0])
    lam_k = np.zeros(x0.shape[0])
    mu_k, p_l = None, None
    x_k = x0
    iteration = 1

    while True:
        f_k = f(x_k)
        f_prime_k = f_grad(x_k).reshape((-1, 1))
        # f_hess_k = f_hessian(x0)

        a_k = grad_ieq(x_k)
        c_k, _ = ieq_conststraints(x_k)

        if iteration > 1 and stop_crit(f_grad, x_k, x0, iteration):
            print("Found solution!!!!")
            return x_k

        # Matrix ak is already defined as transpose in comparison to the book
        p_k, _ = active_set_method(f_hess_k, f_prime_k.T, a_k, -c_k, inequalities=False)
        lam_h = compute_lagrangien_sqp(f_hess_k, a_k, p_k, f_prime_k, c_k)
        p_l = lam_h if p_l is None else (p_l - lam_h)
        mu_k = choose_mu(f_hess_k, f_prime_k, p_k, c_k, mu=mu_k, mu_delta=mu_delta)
        alpha_k = choose_alpha(f=f, fpk=f_prime_k, x=x_k, c=ieq_conststraints, ck=c_k, pk=p_k,
                               eta=eta, mu=mu_k, tau=tau)

        x_k1 = x_k.reshape((-1, 1)) + alpha_k * p_k
        lam_k1 = lam_k + alpha_k * p_l

        f_hess_k = approx_hessian_bfgs(f_hess_k, x_k, x_k1, lam_k1, ieq_conststraints, f_grad, alpha_k, p_k)
        x_k, lam_k = x_k1, lam_k1
        iteration += 1

## Derivative Approximations

In [7]:
def e_i(size, index):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr


# these return the function that computes the gradient, which can then in turn be called to compute the gradient
def approx_grad(f, e=1.1e-8):
    def grad_f(x):
        if x.size == 1:
            return (f(x + e) - f(x)) / e
        return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])

    return grad_f


def approx_hessian(f, e=1.1e-8):
    def hessian_f(x):
        if x.size == 1:
            return (f(x + 2 * e) - 2 * f(x + e) + f(x)) / e ** 2
        return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
            x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
            x)) / e ** 2 for j in range(x.size)] for i in range(x.size)])

    return hessian_f

# Testruns

## Simplex

In [8]:
prob = np.array([[-3], [-2]])
const = np.array([[1, 1], [2, 0.5]])
rhs = np.array([[5], [8]])

x_val, slack_val, sol = simplex_method(linear_problem=prob, constraints=const, constraint_rhs=rhs)
print(f"x = {x_val}")
print(f"slack = {slack_val}")
print(f"solution = {sol}")

Optimal solution found!!!
x = [3.66666667 1.33333333]
slack = [0. 0.]
solution = [[-13.66666667]]


## Generating and solving simplex problems

For the simplex method run 2 chooses an infeasible basis. Run 1 should therefore always give the correct minimum.

In [9]:
# working simplex as placeholder

def simplex(set_b, set_n, A, b, c, print_steps=False):

    A = A.T

    sB = np.zeros((len(set_b), 1))

    xN = np.zeros((len(set_n), 1))

    x = np.zeros((A.shape[0], 1))
    s = np.zeros((A.shape[0], 1))
    
    B = A[set_b].T
    N = A[set_n].T
    cB = c[set_b]
    cN = c[set_n]

    xB = np.linalg.inv(B) @ b

    while True:

        l = np.linalg.inv(B).T @ cB
        
        sN = cN - N.T @ l
        
        if print_steps:
            print(f'\n\nxB:\n{xB}\n\nsN:\n{sN}\n\nlambda:\n{l}\n\nf(x):\n{f(x[:2])}')

        if all(s >= 0 for s in sN):
            optimal_points = [p[0] for p in x]
            return optimal_points

        q = None
        min_change = np.inf
        d = None
        for i, k in enumerate(set_n):

            d_q = np.linalg.inv(B) @ np.reshape(A[k], (-1, 1))
            e = np.identity(c.shape[0] - d_q.shape[0])
            n_q = np.vstack((-d_q, np.reshape(e[i], (-1, 1))))

            change = (c.T @ n_q) / np.linalg.norm(n_q)

            if change < min_change and sN[i] < 0:
                min_change = change
                q = k
                d = d_q

        if all(d_i <= 0 for d_i in d):
            return False

        p = None
        minimum = np.inf
        for i in range(len(d)):
            if d[i] > 0:
                if xB[i] / d[i] < minimum:
                    minimum = xB[i] / d[i]
                    p = i

        x[q] = minimum
              
        xB = xB - d * x[q]  
        x[set_b] = xB[range(len(set_b))]

        xN[set_n.index(q)] = x[q]
        x[set_n] = xN[range(len(set_n))]

        index_n = set_b[p]

        set_b[p] = q
        set_n[set_n.index(q)] = index_n
        
        set_b_len = range(len(set_b))
        set_n_len = range(len(set_n))
        
        B[:, set_b_len] = A[set_b].T
        cB[set_b_len] = c[set_b]
        xB[set_b_len] = x[set_b]
        
        N[:, set_n_len] = A[set_n].T
        cN[set_n_len] = c[set_n]

In [10]:
def generate_problem(n_variables, n_constraints):
    A = np.random.randint(-10, 11, size=(n_constraints, n_variables))
    A = np.concatenate((A, np.identity(n_constraints)), axis=1)
    c = np.random.randint(-10, 11, size=(n_variables, 1))
    c = np.vstack((c, np.zeros((n_constraints, 1))))
    b = np.random.randint(0, 11, size=(n_constraints, 1))

    set_b = [i for i in range(A.shape[1] - n_constraints, A.shape[1])]
    set_n = [i for i in range(A.shape[1] - n_constraints)]

    return A, b, c, set_b, set_n

In [11]:
def generate_and_solve(n_problems, n_variables, n_constraints):
    for i in range(n_problems):
        print(f'PROBLEM {i+1}, RUN 1')

        A, b, c, set_b, set_n = generate_problem(n_variables, n_constraints)

        result = simplex(set_b, set_n, A, b, c)

        while result == False or np.sum(result) < 1e-15:
            A, b, c, set_b, set_n = generate_problem(n_variables, n_constraints)
            result = simplex(set_b, set_n, A, b, c)

        print(f'A:\n{A}\nb:\n{b}\nc:\n{c}\n')
        print(f'Minimum at: {result}\n\n')
        
        print(f'PROBLEM {i+1}, RUN 2')
        
        set_b[-1], set_n[-1] = set_n[-1], set_b[-1]

        result = simplex(set_b, set_n, A, b, c)

        print(f'A:\n{A}\nb:\n{b}\nc:\n{c}\n')
        print(f'Minimum at: {result}\n\n')

In [12]:
generate_and_solve(n_problems=5, n_variables=10, n_constraints=5)

PROBLEM 1, RUN 1
A:
[[10.  4.  1.  3. -2.  8.  0. -4.  8. -8.  1.  0.  0.  0.  0.]
 [-5. -6. -8.  0.  3. 10. -5.  3. -3.  0.  0.  1.  0.  0.  0.]
 [ 7.  4.  5.  6. -5. -2. 10. -2.  7.  3.  0.  0.  1.  0.  0.]
 [10. -4.  3. -7. 10. -4.  4. -9.  9. -6.  0.  0.  0.  1.  0.]
 [ 6. 10.  4.  1. 10. -9.  0.  6. -3. -2.  0.  0.  0.  0.  1.]]
b:
[[6]
 [9]
 [2]
 [5]
 [7]]
c:
[[-6.]
 [ 6.]
 [-7.]
 [-3.]
 [-3.]
 [ 2.]
 [-9.]
 [ 5.]
 [-5.]
 [ 4.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

Minimum at: [0.0, 0.0, 2.477244363091771, 0.32467532467532473, 1.7210749646393206, 2.2443298111323733, 0.8838873499538319, 0.40381063316415683, 0.0, 1.1718112987769365, 0.40783132530120475, 0.0, 0.13227454464682423, 0.5, 0.25]


PROBLEM 1, RUN 2
A:
[[10.  4.  1.  3. -2.  8.  0. -4.  8. -8.  1.  0.  0.  0.  0.]
 [-5. -6. -8.  0.  3. 10. -5.  3. -3.  0.  0.  1.  0.  0.  0.]
 [ 7.  4.  5.  6. -5. -2. 10. -2.  7.  3.  0.  0.  1.  0.  0.]
 [10. -4.  3. -7. 10. -4.  4. -9.  9. -6.  0.  0.  0.  1.  0.]
 [ 6. 10.  4.  1. 10. -9

## Active Set

In [13]:
G = np.array([[2, 0, 0, 0, 0],
              [0, 2, 0, 0, 0],
              [0, 0, 2, 0, 0],
              [0, 0, 0, 2, 0],
              [0, 0, 0, 0, 2]])
c = np.array([-4, -4, 0, 0, 5])
A = np.array([[1, 1, 0, 0, 0], [0, 0, 1, -2, 0]])
b = np.array([[2], [2]])

# feasible_x, W = find_feasible_point(A, b, n_constraints=3, n_variables=2)
# print(feasible_x, W)

result, iterations = active_set_method(G, c, A, b)
print(f"Found solution at {result} after {iterations} iterations")

Optimal solution found!!!
No starting point given: Found one feasible point at '[2. 0. 0. 0. 0.]'


C:\Users\weikl\AppData\Local\Temp\ipykernel_19536\427142891.py:63: RuntimeWarning: divide by zero encountered in true_divide
  piv_weigths = x_B / d_q


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 5 and the array at index 1 has size 1

## Generating and solving QP problems

Regarding the linear constraints: Here they are randomly generated. This is due to us being unsure as to how to construct them from the definition given in the assignment.
<br><br>
Our interpretation was that to make the constraints smooth and linear we would have to split them up as follows:
<br><br>
$|x_1| + |x_2| \leq 1 \implies x_1 + x_2 \leq 1, x_1 - x_2 \leq 1, -x_1 + x_2 \leq 1, -x_1 - x_2 \leq 1$
<br><br>
For twenty variables however, as described in Problem that would lead to $2^20$ constraints, which seems a bit unreasonable

In [14]:
# placeholder asm

def asm(G, W, A, x, c, b, iters=1000):

    for iteration in range(iters):

        A_k = A[W]
        g_k = G @ x + c

        # construct matrix of linear equations to solve for p_k and lambdas
        """
        form: [G, A.T] [p_k] = [-g_k]
              [A,  0 ] [ l ] = [  0 ]
        """
        top = np.hstack((G, A_k.T))
        bottom = np.hstack((A_k, np.zeros((A_k.shape[0], A_k.shape[0]))))
        big = np.vstack((top, bottom))

        # solve
        equation_vec = np.vstack((-g_k, np.zeros((A_k.shape[0], 1))))
        solution = np.linalg.lstsq(big, equation_vec)[0]

        p_k = solution[:G.shape[0]]

        if all(abs(p_i) < 1e-12 for p_i in p_k):

            l_vec = solution[G.shape[0]:]

            if all(l >= 0 for l in l_vec):
                return x

            else:
                j = np.argmin(l_vec)

                W.pop(j)
        else:

            a_k = 1
            blocking_constraints = []

            for i in range(A.shape[0]):

                if i not in W and A[i].T @ p_k > 0:
                    candidate = (b[i] - A[i] @ x) / (A[i] @ p_k)
                    if candidate <= a_k:
                        a_k = candidate
                        blocking_constraints.append(i)

            x = x + a_k * p_k

            if a_k < 1:
                W += [blocking_constraints.pop()]

In [15]:
def generate_and_solve_qp(m_rows=5, M=None, y=None, x0=None):
    
    if M is None:
        M = np.random.randint(-10, 10, size=(m_rows, 2*m_rows))
    
    if y is None:
        y = np.random.randint(-10, 10, size=(m_rows, 1))
    
    G = M.T @ M
    c = -(M.T @ y)
    b = np.ones((M.shape[1], 1), dtype=int)
    
    # for now random constraints
    
    A = np.random.randint(-20, 21, size=(b.shape[0], G.shape[0]))

    n_constraints = b.shape[0]
    n_variables = G.shape[0]
    
    A_simp = np.hstack((A, np.identity(n_constraints)))
    c_simp = np.ones((n_constraints, 1))
    c_simp = np.vstack((np.zeros((n_variables, 1)), c_simp))

    set_b = [i for i in range(A_simp.shape[1] - n_constraints, A_simp.shape[1])]
    set_n = [i for i in range(A_simp.shape[1] - n_constraints)]

    minimiser = simplex(set_b, set_n, A_simp, b, c_simp)

    x = np.reshape(minimiser[:n_variables], (-1, 1))
    z = minimiser[n_variables:]

    W = np.where(A @ x == b)[0].tolist()
    
    result = asm(G, W, A, x, c, b)

    return W, G, A, b, c, x, result

## 3 runs of QP Problem 1

In [16]:
W, G, A, b, c, x, result = generate_and_solve_qp(5)
iters = 1000

tries = 0
while result is None:
    W, G, A, b, c, x, result = generate_and_solve_qp(5)
    tries += 1

print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

x = np.reshape([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.')
    
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

Feasible function found after 0 attempts.
W:
[4, 6, 0, 9, 5, 1, 7]
G:
[[ 252  150  -88   42  156  -68  -20   70  -10   40]
 [ 150  213  -22  -42  113   76   20  108  -22   34]
 [ -88  -22  214 -165  -48  -70   93 -113  -25 -211]
 [  42  -42 -165  166   -9   55 -112   78  -30  165]
 [ 156  113  -48   -9  135  -45   31   22   39   -3]
 [ -68   76  -70   55  -45  314  -70  143 -119  130]
 [ -20   20   93 -112   31  -70   94  -71   68 -111]
 [  70  108 -113   78   22  143  -71  146  -48  158]
 [ -10  -22  -25  -30   39 -119   68  -48  174   13]
 [  40   34 -211  165   -3  130 -111  158   13  262]]
A:
[[-12  20  -8   5  11  -9  18  17 -17   8]
 [  7 -15  -1  13  -5 -14 -19  11  -8 -11]
 [ 14 -18   3  14 -20  -9  17   4  11   0]
 [  4  -2   7 -20   8  -9   5  -8   1  -7]
 [  4 -11  -8   9  12 -12  -1  16  17  -6]
 [-15  17   7  18 -11  16  -6  18   2 -20]
 [ 15 -10 -13  -6 -12  13   4 -20  -8   2]
 [-10 -12 -14   2 -20 -12   6   7 -17 -11]
 [ 14  -3  12  18  -3   9   5  -6  14  20]
 [  0  15

C:\Users\weikl\AppData\Local\Temp\ipykernel_19536\254869811.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  solution = np.linalg.lstsq(big, equation_vec)[0]


## 3 runs of QP Problem 2

In [17]:
W, G, A, b, c, x, result = generate_and_solve_qp(5)
iters = 1000

tries = 0
while result is None:
    W, G, A, b, c, x, result = generate_and_solve_qp(5)
    tries += 1

print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

x = np.reshape([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')
    
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

Feasible function found after 0 attempts.
W:
[3, 7, 9]
G:
[[ 115  -58   51  -91  151   -7  120   71    8  -51]
 [ -58  103 -118   16    9  -12 -122  -73  -79  -87]
 [  51 -118  183    3  -17  -18   99   92  100  140]
 [ -91   16    3  263 -145   88  -17   32  -84   48]
 [ 151    9  -17 -145  317  -33   98   80 -102 -164]
 [  -7  -12  -18   88  -33   75   80   45  -59   15]
 [ 120 -122   99  -17   98   80  288  176  -33   80]
 [  71  -73   92   32   80   45  176  142  -54   70]
 [   8  -79  100  -84 -102  -59  -33  -54  206   74]
 [ -51  -87  140   48 -164   15   80   70   74  278]]
A:
[[-17   4 -14  10 -10   1  19 -19  -3  13]
 [ -3 -10 -16  -8 -15  10  -2  -5 -13 -11]
 [  4 -16 -19 -18   4   1  -5  10  -2 -13]
 [-16 -10  13  14  18   2  19   4   5  -1]
 [ 10  14  -2 -20 -16  -6 -12  14   7  -9]
 [ 18  -1  -1 -13 -17  16 -15   0  18   2]
 [ 13  10 -19  13   6  -2   2 -11  -9  10]
 [ -6 -14 -15  16   9  -1 -16 -15 -13  11]
 [  5  13 -10 -12 -18  17 -19  -8  -6  14]
 [  8   4 -18  19  -1

C:\Users\weikl\AppData\Local\Temp\ipykernel_19536\254869811.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  solution = np.linalg.lstsq(big, equation_vec)[0]


## 4 runs of QP Problem 3

In [ ]:
zero_matrix = np.zeros((2, 4), dtype=int)
M_hat = np.array([[1, 1, 0, 0], [0, 0, 1, 1]])

M_1 = np.hstack((M_hat, zero_matrix, zero_matrix, zero_matrix, zero_matrix))
M_2 = np.hstack((zero_matrix, M_hat, zero_matrix, zero_matrix, zero_matrix))
M_3 = np.hstack((zero_matrix, zero_matrix, M_hat, zero_matrix, zero_matrix))
M_4 = np.hstack((zero_matrix, zero_matrix, zero_matrix, M_hat, zero_matrix))
M_5 = np.hstack((zero_matrix, zero_matrix, zero_matrix, zero_matrix, M_hat))

M = np.vstack((M_1, M_2, M_3, M_4, M_5))
print((M).shape)

y = np.reshape([1, -2, 3, -4, 5, -5, 4, -3, 2, -1], (-1, 1))

W, G, A, b, c, x, result = generate_and_solve_qp(M=M, y=y)

print('\n\nRUN 1\n\n')
print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

print('\n\nRUN 2\n\n')
x = np.reshape([0]*20, (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

print('\n\nRUN 3\n\n')
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')
    
print('\n\nRUN 4\n\n')
x = np.reshape([1]*20, (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

## SQP

In [18]:
rb = Rosenbrock()

sqp_method(rb.f, rb.grad_f, rb.hessian,
           ieq_conststraints=rb.inequality_constraints, grad_ieq=rb.grad_ieqc,
           x0=np.array([1, 1]))

Optimal solution found!!!
No starting point given: Found one feasible point at '[0. 0.]'


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 1